In [34]:
#Reformat the data and pack into an hf5d file
import os
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import random

%matplotlib inline


directory= r'data_gen_reduced_nn_0c/'
runs = []
for filename in os.listdir(directory):
    if filename.startswith('data2c'):
#         run = int(filename.split('_')[2].split('.')[0])
        run = int(filename.split('data2c')[1].split('.')[0])
#         print run
        runs.append(run)
print len(runs)
def import_files(runs):
    import time
    boop=0
    volts=[]
    inputs=[]
    st=time.time()
    st1=time.time()
    lintimes = np.linspace(0,2500,num=100)
    lintimes2 = np.linspace(0,8000,num=100)
#     lintimes3 = np.linspace(0,7051,num=100)
    for run in sorted(runs):
        try:
            if True:
                filename = 'data2c' + str(run) + '.csv'
                data = np.genfromtxt(directory+filename, delimiter=',')
                raw_time=data[1:,0]
                raw_volts=data[1:,1]

#                 filename2 = 'datahalfc' + str(run) + '.csv'
#                 data2 = np.genfromtxt(directory+filename2, delimiter=',')

#                 raw_time2=data2[1:,0]
#                 raw_volts2=data2[1:,1]

#                 filename3 = 'halfc_' + str(run) + '.csv'
#                 data3 = np.genfromtxt(directory+filename3, delimiter=',')

#                 raw_time3=data3[:,0]
#                 raw_volts3=data3[:,1]

                #Impose a linear fit on each data set to ensure the same number of points per trial
                volts.append(np.interp(lintimes,raw_time,raw_volts))
#                 temp_volts1 = np.concatenate((np.interp(lintimes,raw_time,raw_volts),np.interp(lintimes2, raw_time2, raw_volts2)),axis=0)
#                 temp_volts2 = np.interp(lintimes3,raw_time3,raw_volts3)
#                 volts.append(np.concatenate((temp_volts1,temp_volts2),axis=0))
#                 volts = temp_volts1
#                 volts.append(temp_volts1)
                filename2 = 'input'+str(run)+'.csv'
                data2=np.genfromtxt(directory+filename2, delimiter=',')
#                 print data2[1:]
#                 inputs.append(data2[1:28])
                inputs.append(data2)
        except:
            pass

    volts=np.array(volts)
    inputs=np.array(inputs)
#     print volts.shape, inputs.shape
#     print volts.shape[0]
    train = sorted(random.sample(range(volts.shape[0]), int(volts.shape[0]*(3.0/4.))))
    test = [x for x in range(volts.shape[0]) if x not in train]
    P=np.array(inputs)
#     print P.shape
    P_mean = np.mean(inputs)
    P_std = np.std(inputs)
#     P = (P-P_mean)/P_std

    # # Normalize the data
    # P = (P - P.mean())/P.std()
#     print P.shape

    X_train = volts[train,:]
    Y_train = P[train,:]

    X_test = volts[test,:]
    Y_test = P[test,:]

    return [X_train, Y_train, X_test, Y_test]
import h5py
from tqdm import *
def create_database(runs,num,filename,x_dim,y_dim): # Format all the data and create a single databse file
    with h5py.File(filename+".hdf5", "w") as f:
#         help1=import_files(runs[range(len(runs)/num)])
        rowsize1 = 0.75*num
        rowsize2 = num-rowsize1
        dset1 = f.create_dataset("X_train", (rowsize1,x_dim), dtype='f', maxshape=(None,x_dim), chunks=(rowsize1,x_dim), compression="gzip")
        dset2 = f.create_dataset("Y_train", (rowsize1,y_dim), dtype='f', maxshape=(None,y_dim), chunks=(rowsize1,y_dim), compression="gzip")
        dset3 = f.create_dataset("X_test", (rowsize2,x_dim), dtype='f', maxshape=(None,x_dim), chunks=(rowsize2,x_dim), compression="gzip")
        dset4 = f.create_dataset("Y_test", (rowsize2,y_dim), dtype='f', maxshape=(None,y_dim), chunks=(rowsize2,y_dim), compression="gzip")
        leftovers=len(runs)%num
        if leftovers > 0:
            numruns = len(runs)/num+1
        else:
            numruns = len(runs)/num
        for i in tqdm(range(numruns)):
#             print runs[num*i:min(len(runs), (num)+num*i)]
            try:
                help1 = import_files(runs[num*i:num*(i+1)])
                rowsize1 = help1[0].shape[0]
                rowsize2 = help1[2].shape[0]
                rowcount1 = dset1.shape[0]
                rowcount2 = dset3.shape[0]
            except:
                help1 = import_files(runs[num*i:num*i+leftovers])
                rowsize1 = help1[0].shape[0]
                rowsize2 = help1[2].shape[0]
                rowcount1 = dset1.shape[0]
                rowcount2 = dset3.shape[0]

            if i==0:
                dset1[:] = help1[0]
                dset2[:] = help1[1]
                dset3[:] = help1[2]
                dset4[:] = help1[3]
            else:
                dset1.resize(rowcount1 + rowsize1, axis=0 )
                dset2.resize(rowcount1 + rowsize1, axis=0 ) 
                dset3.resize(rowcount2 + rowsize2, axis=0 ) 
                dset4.resize(rowcount2 + rowsize2, axis=0 ) 

                dset1[rowcount1:] = help1[0]
                dset2[rowcount1:] = help1[1]
                dset3[rowcount2:] = help1[2]
                dset4[rowcount2:] = help1[3]
#     print dset1.shape, dset2.shape, dset3.shape, dset4.shape
# dset = X_train
# dset = np.concatenate((dset,X_train),axis=0) 
# print dset1.shape, dset2.shape, dset3.shape, dset4.shape


print 'done'

12000
done


In [29]:
# q=import_files(runs[1:10])
print runs[0:10]

[0 1 2 3 4 5 6 7 8 9]


In [35]:
#1000 takes 50s, 150 takes 30s, 300 takes 30s, 50 takes 28s
# directory= r'Maple_ebdae/data_gen/'
directory = r'data_gen_nn_exp_attempt_2/'

runs = []
for filename in os.listdir(directory):
    if filename.startswith('data2c'):
#         run = int(filename.split('_')[2].split('.')[0])
        run = int(filename.split('data2c')[1].split('.')[0])
#         print run
        runs.append(run)
runs = np.sort(runs)
print len(runs)
filename = 'P2D_data_nn_exp_sort_attempt_2_longtime'
# import_files(runs[1:10])
create_database(runs,100,filename,100,46) # runs, num per round, output dimensions, input features

70000


100%|████████████████████████████████████████████████████████████████████████████████| 700/700 [09:41<00:00,  1.26it/s]


In [54]:
#verify:
f = h5py.File(filename+".hdf5", "r")
X_train = f['X_train']
Y_train = f['Y_train']
X_test = f['X_test']
Y_test = f['Y_test']

print X_train.shape, Y_train.shape, X_test.shape, Y_test.shape
print Y_train[1,21:35]
f.close()

(30995, 200) (30995, 50) (10332, 200) (10332, 50)
[ 110.98213959    9.27678585    0.97082967    0.4901        0.34106874
    4.81682062    4.63150024    4.5303421     4.51685667    4.5021739
    4.4234724     4.34631443    4.27730513    4.23673534]


In [19]:
for i in range(X_train.shape[0]):
    plt.plot(range(X_train.shape[1]),X_train[i])
plt.show()

ValueError: Not a dataset (Not a dataset)